$$
J(θ) = E_τ[R(τ)]
$$
- 最大化所有可能轨迹的期望回报。
- $J(θ) = ∫ P(τ|θ) \cdot R(τ) dτ$
    - $∇_θ J(θ) = ∇_θ ∫ P(τ|θ) \cdot R(τ) dτ$
    - $∇_θ J(θ) = ∫ [∇_θ P(τ|θ)] \cdot R(τ) dτ$
        - 我们没法直接计算 $∇_θ P(τ|θ)$，更重要的是，梯度算子作用在了概率分布 $P(τ|θ)$ 上，这使得我们无法通过采样（sampling）来估计这个积分。我们需要把 $∇_θ$ 从 $P(τ|θ)$ 上“挪开”。
            - $∇_θ P(τ|θ)$ 不是概率分布（可正可负，刻画的是 改变策略参数 $θ$ 时，轨迹 $τ$ 发生的概率会如何变化）
        - $∇_θ P(τ|θ) = P(τ|θ) \cdot ∇_θ \log P(τ|θ)$
    - $∇_θ J(θ) = ∫ P(τ|θ) \cdot [R(τ) \cdot ∇_θ \log P(τ|θ)] dτ$
        - $P(\tau|\theta)$：属于可以被采样的概率分布
        - $∑_t ∇_θ \log π_θ(a_t|s_t)$，这是可以通过策略网络的反向传播计算的
    - $∇_θ J(θ) = E_τ [ R(τ) \cdot ∇_θ \log P(τ|θ)]$
        - $P(τ|θ) = p(s_0) \cdot π_θ(a_0|s_0) \cdot p(s_1|s_0, a_0) \cdot π_θ(a_1|s_1) \cdot ...$
        - $∇_θ \log P(τ|θ) = ∇_θ [ \log p(s_0) + ∑_t \log π_θ(a_t|s_t) + ∑_t \log p(s_{t+1}|s_t, a_t) ]$
        - 环境的状态转移概率 $p(s_{t+1}|s_t, a_t)$ 和初始状态概率 $p(s_0)$ 都与我们的策略参数 $θ$ 无关，所以它们的梯度为0。只剩下策略部分：
            - $∇_θ \log P(τ|θ) = ∑_t ∇_θ \log π_θ(a_t|s_t)$
$$
∇_θ J(θ) = E_τ [ R(τ) \cdot (∑_t ∇_θ \log π_θ(a_t|s_t)) ]
$$

- 整个过程
    - 我们用当前策略 $π_θ$ 运行 N 次，得到 N 条轨迹 $τ_1, τ_2, ..., τ_N$。这等价于从 $P(τ|θ)$ 中采样。
    - 对于每一条轨迹 $τ_i$，我们计算一个值 $V_i = R(τ_i)\cdot∇_θ \log P(τ_i|θ)$
    - 平均：$∇_θ J(θ) ≈ (1/N) * ∑ V_i$。
- 两个可以优化的地方
    - 因果性 (Causality)：在 $t$ 时刻的决策，不应该被 $t$ 时刻之前的奖励所影响。所以，对于 $t$ 时刻的策略梯度 $∇_θ \log π_θ(a_t|s_t)$，我们只应该乘以它未来的回报 $G_t = ∑_{k=t}^T r_k$，而不是整个轨迹的回报 $R(τ)$。
    - 高方差 (High Variance)：回报值本身波动很大。我们可以减去一个基线 (Baseline) $b(s_t)$ 来减小方差，而不改变梯度的期望值。最常用的基线就是状态价值函数 $V(s_t)$。
    - 经过这两个优化，我们将乘数项从 $R(τ)$ 变成了 优势函数 (Advantage Function) $A_t = G_t - V(s_t)$。

| | **变换前 (Problematic Form)** | **变换后 (Solvable Form)** |
| :--- | :--- | :--- |
| **积分形式** | `∫ [∇_θ P(τ\|θ)] * R(τ) dτ` | `∫ P(τ\|θ) * [R(τ) * ∇_θ log P(τ\|θ)] dτ` |
| **采样分布** | `∇_θ P(τ\|θ)` (不是概率分布, **无法采样**) | `P(τ\|θ)` (是概率分布, **可以采样**) |
| **待求值的函数** | `R(τ)` | `R(τ) * ∇_θ log P(τ\|θ)` |
| **可行性** | **无法**通过蒙特卡洛方法估算 | **可以**通过蒙特卡洛方法估算 |

> 我们精心设计一个损失函数 L(θ)，使得最小化 L(θ) 的效果等同于最大化 J(θ)。

$$
L(θ) = - E [ ∑_t (\log π_θ(a_t|s_t) \cdot G_t) ] 
$$

- $∇_θ L(θ) = -∇_θ J(θ)$
    - $∇_θ L(θ)$: `loss.backward()`